# Regression: Splines

Introduction to Machine Learning, BCAM & UPV/EHU course, by Carlos Cernuda, Ekhine Irurozki and Aritz Perez.


## References 

* James, G., Witten, D., Hastie, T., & Tibshirani, R. (2013). 
An introduction to statistical learning (Vol. 112). New York: springer.
* Data sets: http://www-bcf.usc.edu/~gareth/ISL/data.html
* SCIKIT-LEARN library example http://scikit-learn.org
* References Jupyter notebooks:
    - R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016)
    http://www.science.smith.edu/~jcrouser/SDS293/labs/lab10-py.html
    - General Assembly's Data Science course in Washington, DC
    https://github.com/justmarkham/DAT4
    - An Introduction to Statistical Learning (James, Witten, Hastie, Tibshirani, 2013) adapted to Python code
    https://github.com/JWarmenhoven/ISLR-python

## Python Libraries

In [ ]:
##########################################################
import numpy as np #scientific computing (n-dim arrays, etc)
import pandas as pd #data analysis library
##########################################################
# Plots:
import matplotlib.pyplot as plt 
import matplotlib.pylab as pylab
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns #visualization library based on matplotlib
%matplotlib inline
plt.style.use(['seaborn-white'])   
params = {'legend.fontsize': 'xx-large',
              'figure.figsize': (15, 5),
              'axes.labelsize': 'xx-large',
              'axes.titlesize':'xx-large',
              'xtick.labelsize':'xx-large',
              'ytick.labelsize':'xx-large'}    
pylab.rcParams.update(params)  #fix the parameters for the plots

pd.set_option('display.notebook_repr_html', False)
##########################################################
# STATSMODELS: provides classes and functions for the estimation of many different statistical models, 
# as well as for conducting statistical tests, and statistical data exploration.
import statsmodels.api as sm ##
##########################################################
# PATSY: is a Python package for describing statistical models and building design matrices. 
# It is closely inspired by and compatible with the formula mini-language used in R
from patsy import dmatrix 
##########################################################
np.random.seed(0)

## Cubic spline. Basis functions

$$
Y \sim \beta_0+ \beta_1 X + \beta_2 X^2 + \beta_3 X^3
$$

Given data  
$$(y_i,x_i)=(wage_i, age_i) \mbox{ for }i=1,\ldots,n$$

The design matrix is the following:

$$
\left[\begin{array}{c}y_1\\\vdots\\y_n\end{array}\right]
\sim 
\left[\begin{array}{c}
1 & x_1 & x_1^2 & x_1^3\\
  & \vdots  &\\
1 & x_n & x_n^2 & x_n^3
\end{array}\right]
\left[\begin{array}{c}\beta_0\\\beta_1\\\beta_2\\\beta_3\end{array}\right]
$$

In [ ]:
# Cubic Spline without knots (Basis representation)
x_grid = np.linspace(0., 1., 10) #grid points to see the matrix
design_matrix = dmatrix("bs(x, df=3, degree=3)", 
                        {"x": x_grid})
##########################################################
# patsy.dmatrix(formula_like, data={}, return_type='matrix')
# dmatrix module from the patsy library: construct a design matrix given a "formula" and data.

##########################################################
# patsy.bs(x,degree=3, df=None, knots=None, include_intercept=True, lower_bound=None, upper_bound=None)

# The bs() function generates the entire matrix of basis functions for splines with the specified set of knots 
# or degrees of freedom.

# x:      predictor variable
# degree: polynomial degree
# df:     degrees of freedom (columns of the design matrix, number of coefficients)
# knots:  list of knots used
# The spline bases returned by bs() are designed to be compatible with those produced by the R bs function
##########################################################

In [ ]:
print(design_matrix)
# [1 x x^2 x^3]
#3+1 degrees of freedom, no knots point. Cubic polynomial.
#print(design_matrix.design_info)

In [ ]:
# Plot B-spline basis functions (colored curves) and black line (each multiplied by the selected coeff)

#Grid points to plot
x = np.linspace(0., 1., 100)  
design_matrix = dmatrix("bs(x, df=3, degree=3)", 
                        {"x": x})

# Select some coefficients values
beta = np.array([0.1, 1.2, 0.1, 1]) 

# PLOT
plt.figure(figsize=(8, 8))

# Plot the basis functions
plt.plot(x, design_matrix*beta); 

# Plot the spline itself (sum of the basis functions, thick black curve)
plt.plot(x, np.dot(design_matrix, beta), color='k', linewidth=3);

plt.title("Spline basis example (degree=3 and df=4)");

## Data set: Wage (Salary)

Wage data set: 
Salary of a worker in relation with age, sex, etc in 3000 different persons. 

In [ ]:
# Load the data set
wage = pd.read_csv('dataset/Wage.csv')
print(wage.describe())

Given the Wage data we are going to model the following relationship: 
* Age: predictor
* Wage (salary): response.
    
    
$$
wage \sim \beta_0+ \beta_1 age + \beta_2 age^2 + \beta_3 age^3
$$

In [ ]:
#############################
# BUILD THE MODEL: 
# 1) input the data using the design matrix (PREDICTOR) and the RESPONSE
# 2) spline regression formula, fitting
#############################
# Spline formula for: 
# Polynomial degree: 0 (constant), 1 (linear), 2 (quadratic), 3 (cubic)
# Number of knots: 3 at points c_1=25, c_2=40, c_3=60
# Degree of freedom: calculated automatically (number of columns of the design matrix)
#############
design_matrix_data_0 = dmatrix("bs(wage.age, degree=0, knots=(25,40,60), include_intercept=False)",
                          {"wage.age": wage.age}, 
                          return_type='dataframe')
#
design_matrix_data_1 = dmatrix("bs(wage.age, degree=1, knots=(25,40,60), include_intercept=False)",
                          {"wage.age": wage.age}, 
                          return_type='dataframe')
#
design_matrix_data_2 = dmatrix("bs(wage.age, degree=2, knots=(25,40,60), include_intercept=False)",
                          {"wage.age": wage.age}, 
                          return_type='dataframe')
#
design_matrix_data_3 = dmatrix("bs(wage.age, degree=3, knots=(25,40,60), include_intercept=False)",
                          {"wage.age": wage.age}, 
                          return_type='dataframe')
#############
regr_model_0 = sm.GLM(wage.wage, design_matrix_data_0).fit()
regr_model_1 = sm.GLM(wage.wage, design_matrix_data_1).fit()
regr_model_2 = sm.GLM(wage.wage, design_matrix_data_2).fit()
regr_model_3 = sm.GLM(wage.wage, design_matrix_data_3).fit()
#############
# PREDICT WITH THE MODEL:
# Use the model to predict over the new values (grid)
#############
age_grid = np.arange(wage.age.min(), wage.age.max()).reshape(-1,1)
#############
design_matrix_grid_0 = dmatrix("bs(age_grid, degree=0, knots=(25,40,60), include_intercept=False)",
                               {"age_grid": age_grid}, 
                               return_type='dataframe')
design_matrix_grid_1 = dmatrix("bs(age_grid, degree=1, knots=(25,40,60), include_intercept=False)",
                               {"age_grid": age_grid}, 
                               return_type='dataframe')
design_matrix_grid_2 = dmatrix("bs(age_grid, degree=2, knots=(25,40,60), include_intercept=False)",
                               {"age_grid": age_grid}, 
                               return_type='dataframe')
design_matrix_grid_3 = dmatrix("bs(age_grid, degree=3, knots=(25,40,60), include_intercept=False)",
                               {"age_grid": age_grid}, 
                               return_type='dataframe')
#############
pred0 = regr_model_0.predict(design_matrix_grid_0)
pred1 = regr_model_1.predict(design_matrix_grid_1)
pred2 = regr_model_2.predict(design_matrix_grid_2)
pred3 = regr_model_3.predict(design_matrix_grid_3)
#############

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(wage.age, wage.wage, facecolor='None', edgecolor='k', alpha=0.2)

plt.plot(age_grid, pred0, color='b', linewidth=3, label='Constant')
plt.plot(age_grid, pred1, color='g', linewidth=3, label='Linear')
plt.plot(age_grid, pred2, color='k', linewidth=3, label='Quadratic')
plt.plot(age_grid, pred3, color='r', linewidth=3, label='Cubic')

[plt.vlines(i , 0, 350, linestyles='dashed', lw=2, colors='r') for i in [25,40,60]]
plt.legend()
plt.xlim(15,85)
plt.ylim(0,350)
plt.xlabel('age')
plt.ylabel('wage');

In [ ]:
# Coefficients of the Cubic Spline
print(design_matrix_data_3.shape)
#fit3.params
beta_0 = regr_model_3.params['Intercept']
beta_1 = regr_model_3.params[0]
beta_2 = regr_model_3.params[1]
beta_3 = regr_model_3.params[2]
beta_4 = regr_model_3.params[3]
beta_5 = regr_model_3.params[4]
beta_6 = regr_model_3.params[5]

print(beta_0,beta_1,beta_2,beta_3,beta_4,beta_5,beta_6)

## Knots number and locations

The regression spline is more flexible in regions with more knots, because the polynomial coefficients can change rapidly and adapt better the data shape. The idea it to place more knots where the function might vary rapidly and less knots where is more stable. 

However, in practice is common to place knots using an uniform grid. One possible way to do that is to, instead of fixing the knots, fix the degrees of freedom and then the python function will automatically place the corresponding number of knots at uniform quantiles of the data. 


In [ ]:
#############################
# Cubic spline specifying degrees of freedom
#############################
# df = 1 + 3 + number of knots (python 3 + k)
#######
# Build the model:
design_matrix_data_3_6 = dmatrix("bs(wage.age, degree=3, df=6, include_intercept=False)",
                        {"wage.age": wage.age}, 
                         return_type='dataframe')

regr_model_3_6 = sm.GLM(wage.wage, design_matrix_data_3_6).fit()
#######
# Predict with the model:
design_matrix_grid_3_6 = dmatrix("bs(age_grid, degree=3, df=6, include_intercept=False)",
                             {"age_grid": age_grid}, 
                             return_type='dataframe')
pred3_6 = regr_model_3_6.predict(design_matrix_grid_3_6)
#######
print(design_matrix_grid_3_6.shape)
#fit3_6.params
#Python chooses 3 knots which correspond to the 25th, 50th, and 75th percentiles of age.
#DF = Number of columns in the design matrix
#############################

In [ ]:
#PLOTS
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(12,5))
#######
# Left
ax1.scatter(wage.age, wage.wage, facecolor='None', edgecolor='k', alpha=0.2)
ax1.plot(age_grid, pred3, color='r', linewidth=3)
[ax1.vlines(i , 0, 350, linestyles='dashed', lw=2, colors='r') for i in [25,40,60]]

ax1.set_xlim(15,85)
ax1.set_ylim(0,350)
ax1.set_xlabel('age')
ax1.set_ylabel('wage')
ax1.set_title('Cubic Knots Fixed');
#######
# Right
ax2.scatter(wage.age, wage.wage, facecolor='None', edgecolor='k', alpha=0.2)
ax2.plot(age_grid, pred3_6, color='b', linewidth=3)
[ax2.vlines(i , 0, 350, linestyles='dashed', lw=2, colors='b') for i in [33.75,42,51]]#from describe data frame

ax2.set_xlim(15,85)
ax2.set_ylim(0,350)
ax2.set_xlabel('age')
ax2.set_ylabel('wage')
ax2.set_title('Cubic Df Fixed');

Try different knots and degrees of freedom and see how the curves change. 

A more objective approach is to use cross-validation. Remove a portion of the data, fit a spline with the remaining data, and then use the spline to make predictions over the separated data. Repeat this process multiple times until each observation has been left out once, and then compute the overall cross-validated RSS. The procedure can be repeated with different number of knots and the value given the smallest RSS is chosen. 

## Splines vs Polynomials

In [ ]:
#############################
# Splines versus Polynomials
#############################
# Spline regression degree=3 df=10 (10-3 = 7 knots)
#######
# Fit the model
design_matrix_data_5 = dmatrix("bs(wage.age, degree=3, df=10, include_intercept=False)",
                          {"wage.age": wage.age}, 
                          return_type='dataframe')
regr_model_5 = sm.GLM(wage.wage, design_matrix_data_5).fit()
#######
# Predict 
age_grid = np.arange(wage.age.min(), wage.age.max()).reshape(-1,1)
design_matrix_grid_5 = dmatrix("bs(age_grid, degree=3, df=10, include_intercept=False)",
                               {"age_grid": age_grid}) 
pred5 = regr_model_5.predict(design_matrix_grid_5)
#############################
# Plot
plt.figure(figsize=(8,8))
plt.scatter(wage.age, wage.wage, facecolor='None', edgecolor='k', alpha=0.2)
plt.plot(age_grid, pred5, color='r', linewidth=3, label='Spline')
#######
# Polynomial regression degree=10
sns.regplot(wage.age, wage.wage, order=10, ci=None, scatter=False, label='Polynomial')                               
#######                      
plt.legend()
plt.xlim(15,85)
plt.ylim(0,350)
plt.xlabel('age')
plt.ylabel('wage')
plt.title('Spline (7 knots) vs Polynomial (degree 10)');